<a href="https://colab.research.google.com/github/KavithaR14/self-evaluation/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pyngrok
!pip install matplotlib
!pip install reportlab
!pip install fpdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=adcaafe2b872c549203ce80471a5f9cf0a62f22d635afb7a3c896cb1c444ef0a
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
from flask import Flask, render_template_string, request, redirect, url_for, session,send_file
from pyngrok import ngrok
import io
import base64
import matplotlib.pyplot as plt
import logging
from flask import send_file
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from io import BytesIO
from fpdf import FPDF


app = Flask(__name__)
app.secret_key = 'your_secret_key'


html_start = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Innovative Front Page</title>
    <style>
        @import url('https://fonts.googleapis.com/css?family=Montserrat:400,700');
        @keyframes backgroundTransition {
    from {
        background: linear-gradient(to right, #2e096e, #4af1e4, #d365fb); /* Starting gradient background */
    }
    to {
        background: linear-gradient(to right, #4af1e4, #d365fb, #2e096e); /* Final gradient background */
    }
}

.element {
    animation: backgroundTransition 5s infinite; /* Apply animation to the desired element */
}




        @keyframes fadeIn {
            from {
                opacity: 0;
            }
            to {
                opacity: 1;
            }
        }

        @keyframes slideIn {
            from {
                transform: translateY(-50px);
                opacity: 0;
            }
            to {
                transform: translateY(0);
                opacity: 1;
            }
        }

        @keyframes pulse {
            0% {
                transform: scale(1);
            }
            50% {
                transform: scale(1.05);
            }
            100% {
                transform: scale(1);
            }
        }

        @keyframes colorChange {
            0% {
                color: #ffffff;
            }
            25% {
                color: #4ff3ff;
            }
            50% {
                color: #5b4fff;
            }
            75% {
                color: #ff4fb6;
            }
            100% {
                color: #ffffff;
            }
        }

        body {
            font-family: 'Montserrat', sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background: #ffffff; /* Initial white background */
            animation: backgroundTransition 5s ease-out infinite;
        }

        .container {
            background: #fff;
            align-items: center;
            justify-content: center;
            display: flex;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 14px 28px rgba(0, 0, 0, 0.25),
                        0 10px 10px rgba(0, 0, 0, 0.22);
            position: relative;
            overflow: hidden;
            width: 780px;
            max-width: 100%;
            min-height: 600px;
            opacity: 0;
            animation: fadeIn 1.5s forwards;
        }

        h1 {
            font-size: 3rem;
            margin-bottom: 20px;
            color: #ffffff; /* Initial color */
            animation: colorChange 5s infinite, slideIn 1s ease-out;
        }

        p {
            font-size: 1.2rem;
            margin-bottom: 40px;
            animation: fadeIn 2s ease-out;
        }

        .btn {
            display: inline-block;
            padding: 15px 30px;
            font-size: 1rem;
            font-weight: bold;
            color: #ffffff;
            background-color: #6a0dad;
            border: none;
            border-radius: 5px;
            text-transform: uppercase;
            text-decoration: none;
            transition: background-color 0.3s ease, transform 0.3s ease;
            animation: pulse 1.5s infinite;
        }

        .btn:hover {
            background-color: #b93dff;
            transform: scale(1.05);
        }
    </style>
</head>
<body>
    <div class="container">
        <center>
            <h1>Welcome to Our<br/>Self Evaluation Platform</h1><br><br>
            <p>Self-evaluation allows you to understand your strengths and weaknesses.<br> 🚀Let's get started on your journey to success.</p><br><br>
            <a href="{{ url_for('index') }}" class="btn">Get Started ➡️</a>
        </center>
    </div>
</body>
</html>

"""
html_index="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login and Sign Up Page</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background: linear-gradient(to right,  #2e096e, #4af1e4,#d365fb);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .container {
            background: #fff;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 14px 28px rgba(0, 0, 0, 0.25),
                        0 10px 10px rgba(0, 0, 0, 0.22);
            position: relative;
            overflow: hidden;
            width: 780px;
            max-width: 200%;
            min-height: 600px;
        }
        .form-container {
            position: absolute;
            top: 0;
            height: 100%;
            transition: all 0.6s ease-in-out;
        }
        .login-container {
            left: 0;
            width: 50%;
            z-index: 2;
        }
        .sign-up-container {
            left: 0;
            width: 50%;
            opacity: 0;
            z-index: 1;
        }
        .container.right-panel-active .login-container {
            transform: translateX(100%);
        }
        .container.right-panel-active .sign-up-container {
            transform: translateX(100%);
            opacity: 1;
            z-index: 5;
            animation: show 0.6s;
        }
        @keyframes show {
            0%, 49.99% {
                opacity: 0;
                z-index: 1;
            }
            50%, 100% {
                opacity: 1;
                z-index: 5;
            }
        }
        .overlay-container {
            position: absolute;
            top: 0;
            left: 50%;
            width: 50%;
            height: 100%;
            overflow: hidden;
            transition: transform 0.6s ease-in-out;
            z-index: 100;
        }
        .container.right-panel-active .overlay-container {
            transform: translateX(-100%);
        }
        .overlay {
            background: #6a0dad;
            background: -webkit-linear-gradient(to right, #6a0dad, #b93dff);
            background: linear-gradient(to right, #6a0dad, #b93dff);
            background-repeat: no-repeat;
            background-size: cover;
            background-position: 0 0;
            color: #ffffff;
            position: relative;
            left: -100%;
            height: 100%;
            width: 200%;
            transform: translateX(0);
            transition: transform 0.6s ease-in-out;
        }
        .container.right-panel-active .overlay {
            transform: translateX(50%);
        }
        .overlay-panel {
            position: absolute;
            display: flex;
            align-items: center;
            justify-content: center;
            flex-direction: column;
            padding: 0 40px;
            text-align: center;
            top: 0;
            height: 100%;
            width: 50%;
            transform: translateX(0);
            transition: transform 0.6s ease-in-out;
        }
        .overlay-left {
            transform: translateX(-20%);
        }
        .container.right-panel-active .overlay-left {
            transform: translateX(0);
        }
        .overlay-right {
            right: 0;
            transform: translateX(0);
        }
        .container.right-panel-active .overlay-right {
            transform: translateX(20%);
        }
        form {
            background-color: #ffffff;
            display: flex;
            flex-direction: column;
            padding: 0 50px;
            height: 100%;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        form h1 {
            font-weight: bold;
            margin: 0;
        }
        form input {
            background-color: #eee;
            border: none;
            border-radius: 10px;
            border: none;
            padding: 12px 15px;
            margin: 8px 0;
            width: 100%;
        }
        form button {
            border-radius: 20px;
            border: 1px solid #6a0dad;
            background-color: #6a0dad;
            color: #ffffff;
            font-size: 15px;
            font-weight: bold;
            padding: 12px 45px;
            letter-spacing: 1px;
            text-transform: uppercase;
            transition: transform 80ms ease-in;
        }
        form button:hover {
            background-color: #b93dff;
        }
        form a {
            color: #333;
            font-size: 14px;
            text-decoration: none;
            margin: 15px 0;
        }
        .ghost {
            background-color: #ffffff;
            border-color: #e9d6d6;
            border-radius: 10%;
        }
        .ghost:hover {
            background-color: #f1f1f1;
        }
    </style>
</head>
<body>
    <div class="container" id="container">
        <div class="form-container sign-up-container">
            <form id="signUpForm" action="{{ url_for('signUp') }}" method="post">
                <h1>Create Account</h1><br><br>
                <input type="text" placeholder="Name" name="username" required />
                <input type="email" placeholder="Email" name="email" required />
                <input type="password" placeholder="Password" name="password" required /><br>
                <button type="submit">Sign Up</button>
            </form>
        </div>
        <div class="form-container login-container">
            <form id="loginForm" action="{{ url_for('login') }}" method="post">
                <div>
                <h5><b><u>EVALUATE YOURSELF</u></b></h5>
                <h3 style="text-transform: uppercase;"><i><q>understanding yourself<br> is a key to<br> enhance yourself</q></i></h3><br><br>
                </div>
                <h1>Login</h1><br><br>
                <input type="text" placeholder="Username" name="username" required />
                <input type="password" placeholder="Password" name="password" required />
                <a href="{{ url_for('resetpassword') }}"><u>Forgot your password?</u></a><br>
                <button type="submit">Login</button>
            </form>
        </div>
        <div class="overlay-container">
            <div class="overlay">
                <div class="overlay-panel overlay-left">
                    <h1 style="text-align: left; color: #ffffff"><b>Welcome Back!🤝</b></h1><br><br>
                    <p style="font-size: xlarge;">To keep connected with us please<br/>login with your account</p><br>
                    <button class="ghost" id="login"><h2 style="text-align: center;">Login</h2></button>
                </div>
                <div class="overlay-panel overlay-right">
                    <h1 style="color: #ffffff">Hello, Friend!👋</h1><br><br>
                    <p style="font-size: xlarge;">Enter your personal details and <br/>start journey with us</p><br>
                    <button class="ghost" id="signUp"><h2 style="text-align: center;">Sign Up</h2></button>
                </div>
            </div>
        </div>
    </div>

    <script>
    document.addEventListener('DOMContentLoaded', function () {
        let users = JSON.parse(localStorage.getItem('users')) || {};

        const signUpButton = document.getElementById('signUp');
        const loginButton = document.getElementById('login');
        const container = document.querySelector('.container');

        signUpButton.addEventListener('click', () => {
            container.classList.add('right-panel-active');
        });

        loginButton.addEventListener('click', () => {
            container.classList.remove('right-panel-active');
        });

        // Handle the sign-up form submission
        const signUpForm = document.getElementById('signUpForm');

        signUpForm.addEventListener('submit', function(event) {
            event.preventDefault();

            const username = document.querySelector('input[name="username"]').value;
            const email = document.querySelector('input[name="email"]').value;
            const password = document.querySelector('input[name="password"]').value;

            // Email validation function
            function validateEmail(email) {
                const emailRegex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
                return emailRegex.test(email);
            }

            // Password validation function
            function validatePassword(password) {
                if (password.length < 8) {
                    alert('Password must be at least 8 characters long');
                    return false;
                }
                if (!/[a-z]/.test(password)) {
                    alert('Password must contain at least one lowercase letter');
                    return false;
                }
                if (!/[A-Z]/.test(password)) {
                    alert('Password must contain at least one uppercase letter');
                    return false;
                }
                if (!/[0-9]/.test(password)) {
                    alert('Password must contain at least one number');
                    return false;
                }
                if (!/[!@#$%^&*(),.?":{}|<>]/.test(password)) {
                    alert('Password must contain at least one special character');
                    return false;
                }
                if (/\s/.test(password)) {
                    alert('Password cannot contain spaces');
                    return false;
                }
                const common_passwords = ["password", "12345678", "qwerty", "letmein"];
                if (common_passwords.includes(password.toLowerCase())) {
                    alert('Password is too common');
                    return false;
                }
                return true;
            }

            // Validate email and password before submitting
            if (validateEmail(email) && validatePassword(password)) {
                const newUser = {
                    username: username,
                    email: email,
                    password: password
                };

                // Use Fetch API to POST data to db.json
                fetch('http://localhost:3000/users', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify(newUser)
                })
                .then(response => response.json())
                .then(data => {
                    console.log("Account created successfully!", data);
                    alert("Account created successfully!👍");
                    signUpForm.submit();
                })
                .catch(error => {
                    console.error("Error:", error);
                    alert("Login to continue!😊");
               });
            }
        });

        // Handle the login form submission
        const loginForm = document.getElementById('loginForm');

        loginForm.addEventListener('submit', function(event) {
            event.preventDefault();

            const username = document.querySelector('input[name="username"]').value;
            const password = document.querySelector('input[name="password"]').value;

            // Fetch the existing users from db.json
            fetch('http://localhost:3000/users')
            .then(response => response.json())
            .then(users => {
                const user = users.find(user => user.username === username && user.password === password);
                if (user) {
                    console.log("Login successful!");
                    alert("Login successful!");
                    loginForm.submit();
                } else {
                    console.log("Invalid username or password");
                    alert("Invalid username or password");
                }
            })
            .catch(error => {
                console.error("Error:", error);
                alert("Failed to log in.");
            });
        });
    });
</script>


</body>
</html>
"""




# HTML code for the description page
html_description = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Self Evaluation</title>
    <style>
        @import url('https://fonts.googleapis.com/css?family=Montserrat:400,700');
        @keyframes backgroundTransition {
    from {
        background: linear-gradient(to right, #2e096e, #4af1e4, #d365fb); /* Starting gradient background */
    }
    to {
        background: linear-gradient(to right, #4af1e4, #d365fb, #2e096e); /* Final gradient background */
    }
}

.element {
    animation: backgroundTransition 5s infinite; /* Apply animation to the desired element */
}

        body {
            font-family: 'Montserrat', sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background: #ffffff; /* Initial white background */
            animation: backgroundTransition 5s ease-out infinite;
        }
        body {
            font-family: 'Arial', sans-serif;
            background: linear-gradient(to right,  #2e096e, #4af1e4,#d365fb);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background-size: cover;
        }

        .container {
            background: white;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 14px 28px rgba(0, 0, 0, 0.25),
                        0 10px 10px rgba(0, 0, 0, 0.22);
            position: center;
            width: 780px;
            max-width: 200%;
            min-height: 600px;
        }

        h2, h3, p {
            margin: 20px 0;
        }
        ul {
            list-style-type: none;
            padding: 0;
        }
        li {
            margin: 5px 0;
        }
        .form-group {
            text-align: center;
            margin-top: 20px;
        }
        .cta-button {
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            background-color: green;
            color: white;
            font-size: 16px;
            cursor: pointer;
            display: inline-block;
            text-decoration: none;
        }
        .cta-button:hover {
            background-color: rgb(59, 199, 59);
        }
    </style>
</head>
<body>
<div class="container">
    <center>
        <h2><b>📝Self Evaluation✅</b></h2>
    </center>
    <br>
        <h3><b>Description</b></h3>
        <p>➤ In this self-analysis test, you will answer all the given questions. This test will help you analyze your mental stability and character. The scores you receive will indicate your levels of empathy, adaptability, and communication. By the end of this assessment, you will have a better understanding of yourself.</p>
        <p>➤ In this assessment, we have categorized some topics to help you answer the questions. The different categories are listed below:</p>
        <br><h3><b>Categories</b></h3>
        <ul>
            <li>1. <b>Empathy🎭</b>:
            Questions in this category will help assess your ability to understand and share the feelings of others.</li>
            <br><li>2. <b>Adaptability🔀</b>:
            This section measures your ability to adjust to new conditions and challenges.</li>
            <br><li>3. <b>Communication🔊:</b>
            These questions evaluate your effectiveness in conveying information and understanding others.</li>
        </ul><br><br>
    <div class="form-group">
        <a href="{{ url_for('empathy') }}" class="cta-button">Start Assessment</a>
    </div>
</div>
</body>
</html>
"""
html_empathy = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Assessment Page</title>
    <style>
        /* Your existing styles */
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            height: 100vh;
            box-sizing: border-box;
            background-color: #f5f5f5;
        }
        .split {
            flex: 1;
            display: flex;
            padding: 20px;
            box-sizing: border-box;
        }
        .left {
            width: 20%;
            background-color: blueviolet;
            color: #fff;
            position: fixed;
            height: 100vh;
            overflow-y: auto;
            padding: 20px;
            text-decoration: none;
            display: inline-block;
        }
        .right {
            background-color: #fff;
            padding-left: 40px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            margin-left: 20%;
        }
        .left h3, .right h2 {
            margin: 0;
            padding: 10px 0;
            border-bottom: 2px solid #fff;
            text-align: center;
        }
        .left ul {
            list-style-type: none;
            padding: 0;
            margin: 0;
            color: #fff;
            text-decoration: none;
        }
        .left li {
            padding: 10px;
            border-bottom: 1px solid #555;
        }
        .left a {
            color: white;
            text-decoration: none;
            display: block;
        }
        .left li:hover {
            background-color: #444;
        }
        .question-container {
            flex-grow: 1;
            display: flex;
            flex-direction: column;
            justify-content: center;
        }
        .question {
            margin-bottom: -2%;
        }
        .options {
            list-style-type: none;
            padding: 0;
        }
        .options li {
            margin: 10px 0;
        }
        .options input {
            margin-right: 10px;
        }
        .form-group {
            display: flex;
            justify-content: space-between;
            padding: 20px 0;
        }
        .nav-button {
            padding: 10px 20px;
            background-color: #28a745;
            border: none;
            border-radius: 4px;
            color: white;
            font-size: 16px;
            cursor: pointer;
            text-decoration: none;
        }
        .nav-button:hover {
            background-color: #218838;
        }
        .questions {
            display: none;
            padding: 10px;
            margin-left: 20px;
            margin-top: 10px;
        }
        .questions span {
            display: inline-block;
            width: 25px;
            height: 30px;
            line-height: 30px;
            margin: 5px;
            text-align: center;
            background-color: red;
            color: white;
            border-radius: 50%;
            cursor: pointer;
        }
        .questions span.answered {
            background-color: green;
        }
    </style>
</head>
<body>
    <div class="split left">
        <h3>Categories</h3>
        <ul>
           <li onclick="toggleQuestions('empathy')">Empathy</li>
            <div id="empathy" class="questions">
                <span id="q1">1</span><span id="q2">2</span><span id="q3">3</span><span id="q4">4</span><span id="q5">5</span>
                <span id="q6">6</span><span id="q7">7</span><span id="q8">8</span><span id="q9">9</span><span id="q10">10</span>
            </div>
            <li onclick="toggleQuestions('adaptability')">Adaptability</li>
            <div id="adaptability" class="questions">
            </div>
            <li onclick="toggleQuestions('communication')">Communication</li>
            <div id="communication" class="questions">
            </div>
        </ul>
    </div>

    <div class="split right">
        <div>
            <h2>Empathy Questions</h2>
            <form action="/empathy" method="POST">
            <div class="question-container">
                <!-- Your question blocks -->
                 <div class="question" id="question-1">
                        <p>1. I actively listen to others and try to understand their feelings.</p>
                 </div>
                        <ul class="options">
                            <li><label><input type="radio" name="q1" value="Strongly Disagree" onclick="saveAndMarkAnswer(1, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q1" value="Disagree" onclick="saveAndMarkAnswer(1, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q1" value="Neutral" onclick="saveAndMarkAnswer(1, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q1" value="Agree" onclick="saveAndMarkAnswer(1, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q1" value="Strongly Agree" onclick="saveAndMarkAnswer(1, 'Strongly Agree')"> Strongly Agree</label></li>
                        </ul>

                <div class="question" id="question-2">
                    <p>2. I can easily put myself in someone else's shoes.</p>
                </div>
                       <ul class="options">
                            <li><label><input type="radio" name="q2" value="Strongly Disagree" onclick="saveAndMarkAnswer(2, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q2" value="Disagree" onclick="saveAndMarkAnswer(2, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q2" value="Neutral" onclick="saveAndMarkAnswer(2, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q2" value="Agree" onclick="saveAndMarkAnswer(2, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q2" value="Strongly Agree" onclick="saveAndMarkAnswer(2, 'Strongly Agree')"> Strongly Agree</label></li>
                       </ul>
                <div class="question">
                    <p>3.I am sensitive to the moods and feelings of those around me.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q3" value="Strongly Disagree" onclick="saveAndMarkAnswer(3, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q3" value="Disagree" onclick="saveAndMarkAnswer(3, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q3" value="Neutral" onclick="saveAndMarkAnswer(3, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q3" value="Agree" onclick="saveAndMarkAnswer(3, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q3" value="Strongly Agree" onclick="saveAndMarkAnswer(3, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>4.I offer support and help when I see someone in distress.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q4" value="Strongly Disagree" onclick="saveAndMarkAnswer(4, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q4" value="Disagree" onclick="saveAndMarkAnswer(4, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q4" value="Neutral" onclick="saveAndMarkAnswer(4, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q4" value="Agree" onclick="saveAndMarkAnswer(4, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q4" value="Strongly Agree" onclick="saveAndMarkAnswer(4, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>5.I often think about how my actions affect others.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q5" value="Strongly Disagree" onclick="saveAndMarkAnswer(5, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q5" value="Disagree" onclick="saveAndMarkAnswer(5, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q5" value="Neutral" onclick="saveAndMarkAnswer(5, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q5" value="Agree" onclick="saveAndMarkAnswer(5, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q5" value="Strongly Agree" onclick="saveAndMarkAnswer(5, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>6.I am easily moved by other's suffering and feel genuine sadness for them.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q6" value="Strongly Disagree" onclick="saveAndMarkAnswer(6, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q6" value="Disagree" onclick="saveAndMarkAnswer(6, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q6" value="Neutral" onclick="saveAndMarkAnswer(6, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q6" value="Agree" onclick="saveAndMarkAnswer(6, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q6" value="Strongly Agree" onclick="saveAndMarkAnswer(6, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>7.I can see or hear someone who is happy, it makes me feel happy too.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q7" value="Strongly Disagree" onclick="saveAndMarkAnswer(7, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q7" value="Disagree" onclick="saveAndMarkAnswer(7, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q7" value="Neutral" onclick="saveAndMarkAnswer(7, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q7" value="Agree" onclick="saveAndMarkAnswer(7, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q7" value="Strongly Agree" onclick="saveAndMarkAnswer(7, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>8.I can easily sense if the other person is under stress.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q8" value="Strongly Disagree" onclick="saveAndMarkAnswer(8, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q8" value="Disagree" onclick="saveAndMarkAnswer(8, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q8" value="Neutral" onclick="saveAndMarkAnswer(8, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q8" value="Agree" onclick="saveAndMarkAnswer(8, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q8" value="Strongly Agree" onclick="saveAndMarkAnswer(8, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>9.I can relate with the feelings of the characters in a novel/ story with my life.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q9" value="Strongly Disagree" onclick="saveAndMarkAnswer(9, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q9" value="Disagree" onclick="saveAndMarkAnswer(9, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q9" value="Neutral" onclick="saveAndMarkAnswer(9, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q9" value="Agree" onclick="saveAndMarkAnswer(9, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q9" value="Strongly Agree" onclick="saveAndMarkAnswer(9, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
                <div class="question">
                    <p>10.I go to the extra mile to help and understand others problem.</p>
                </div>
                <ul class="options">
                            <li><label><input type="radio" name="q10" value="Strongly Disagree" onclick="saveAndMarkAnswer(10, 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="q10" value="Disagree" onclick="saveAndMarkAnswer(10, 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="q10" value="Neutral" onclick="saveAndMarkAnswer(10, 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="q10" value="Agree" onclick="saveAndMarkAnswer(10, 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="q10" value="Strongly Agree" onclick="saveAndMarkAnswer(10, 'Strongly Agree')"> Strongly Agree</label></li>
                </ul>
              </div>
            </div>
       <div class="form-group">
         <a href="/description" class="nav-button">Previous</a>
         <input type="submit" value="Next" class="nav-button">
      </form>
    </div>
    </div>
    <script>
       document.addEventListener('DOMContentLoaded', function() {
        // Restore responses when the page loads
        restoreResponses();

        // Add event listener to the Previous button
        const prevButton = document.querySelector('.form-group .nav-button');
        if (prevButton) {
            prevButton.addEventListener('click', function(event) {
                event.preventDefault(); // Prevent default link behavior
                saveResponsesAndNavigate(this.getAttribute('href')); // Navigate after saving
            });
        }
    });

    function toggleQuestions(categoryId) {
        const questionsDiv = document.getElementById(categoryId);
        const isVisible = questionsDiv.style.display === 'block';
        document.querySelectorAll('.questions').forEach(div => div.style.display = 'none');
        questionsDiv.style.display = isVisible ? 'none' : 'block';

        if (!isVisible) {
            markCategoryAsAnswered(categoryId);
        }
    }

    function markCategoryAsAnswered(categoryId) {
        const categoryElement = document.querySelector(`.left li:nth-child(${getCategoryIndex(categoryId)})`);
        if (categoryElement) {
            categoryElement.classList.add('answered');
        }
    }

    function getCategoryIndex(categoryId) {
        const categories = ['empathy', 'adaptability', 'communication'];
        return categories.indexOf(categoryId) + 1;
    }

    function saveResponses() {
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            if (option.checked) {
                localStorage.setItem(option.name, option.value);
            }
        });
    }

    function restoreResponses() {
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            const savedValue = localStorage.getItem(option.name);
            if (savedValue && option.value === savedValue) {
                option.checked = true;
                markQuestionAsAnswered(option.name);
                markCategoryAsAnsweredFromQuestion(option.name);
            }
        });
    }

    function markQuestionAsAnswered(questionName) {
        const questionSpan = document.getElementById(questionName);
        if (questionSpan) {
            questionSpan.classList.add('answered');
        }
    }

    function markCategoryAsAnsweredFromQuestion(questionName) {
        const questionNumber = questionName.replace('q', '');
        const categoryId = getCategoryFromQuestion(questionNumber);
        markCategoryAsAnswered(categoryId);
    }

    function getCategoryFromQuestion(questionNumber) {
        if (questionNumber <= 10) {
            return 'empathy';
        } else if (questionNumber <= 20) {
            return 'adaptability';
        } else {
            return 'communication';
        }
    }

    function saveAndMarkAnswer(questionNumber, answer) {
        saveResponses();
        const questionSpan = document.getElementById(`q${questionNumber}`);
        if (questionSpan) {
            questionSpan.classList.add('answered');
            const categoryId = getCategoryFromQuestion(questionNumber);
            markCategoryAsAnswered(categoryId);
        }
    }

    function saveResponsesAndNavigate(url) {
        saveResponses();
        window.location.href = url;
    }
    </script>
</body>
</html>

"""
# Display the HTML
from IPython.display import HTML


# HTML code as a string


html_adaptability= """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Assessment Page</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            height: 100vh;
            box-sizing: border-box;
            background-color: #f5f5f5;
        }
        .split {
            flex: 1;
            display: flex;
            padding: 20px;
            box-sizing: border-box;
        }
        .left {
            width: 20%;
            background-color: blueviolet;
            color: #fff;
            position: fixed;
            height: 100vh;
            overflow-y: auto;
            padding: 20px;
            text-decoration: none;
            display: inline-block;
        }
        .right {
            background-color: #fff;
            padding-left: 40px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            margin-left: 20%;
        }
        .left h3, .right h2 {
            margin: 0;
            padding: 10px 0;
            border-bottom: 2px solid #fff;
            text-align: center;
        }
        .left ul {
            list-style-type: none;
            padding: 0;
            margin: 0;
            color: #fff;
            text-decoration: none;
        }
        .left li {
            padding: 10px;
            border-bottom: 1px solid #555;
        }
        .left a {
            color: white;
            text-decoration: none;
            display: block;
        }
        .left li:hover {
            background-color: #444;
        }
        .question-container {
            flex-grow: 1;
            display: flex;
            flex-direction: column;
            justify-content: center;
        }
        .question {
            margin-bottom: -2%;
        }
        .options {
            list-style-type: none;
            padding: 0;
        }
        .options li {
            margin: 10px 0;
        }
        .options input {
            margin-right: 10px;
        }
        .form-group {
            display: flex;
            justify-content: space-between;
            padding: 20px 0;
        }
        .nav-button {
            padding: 10px 20px;
            background-color: #28a745;
            border: none;
            border-radius: 4px;
            color: white;
            font-size: 16px;
            cursor: pointer;
            text-decoration: none;
        }
        .nav-button:hover {
            background-color: #218838;
        }
        .questions {
            display: none;
            padding: 10px;
            margin-left: 20px;
            margin-top: 10px;
        }
        .questions span {
            display: inline-block;
            width: 25px;
            height: 30px;
            line-height: 30px;
            margin: 5px;
            text-align: center;
            background-color: red;
            color: white;
            border-radius: 50%;
            cursor: pointer;
        }
        .questions span.answered {
            background-color: green;
        }
    </style>
</head>
<body>
    <div class="split left">
        <h3>Categories</h3>
        <ul>
            <li onclick="toggleQuestions('empathy')">Empathy</li>
            <div id="empathy" class="questions">
            </div>
            <li onclick="toggleQuestions('adaptability')">Adaptability</li>
            <div id="adaptability" class="questions">
                <span id="adaptability_q1">1</span><span id="adaptability_q2">2</span><span id="adaptability_q3">3</span><span id="adaptability_q4">4</span><span id="adaptability_q5">5</span>
                <span id="adaptability_q6">6</span><span id="adaptability_q7">7</span><span id="adaptability_q8">8</span><span id="adaptability_q9">9</span><span id="adaptability_q10">10</span>
            </div>
            <li onclick="toggleQuestions('communication')">Communication</li>
            <div id="communication" class="questions">
            </div>
       </ul>
    </div>

    <div class="split right">
        <div>
            <h2>Adaptability Questions</h2>
            <form action="/adaptability" method="POST">
                <div class="question-container">
                    <!-- Questions and options go here -->
                    <!-- Example for the first question: -->
                    <div class="question">
                        <p>1. I embrace change and see it as an opportunity for growth.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q1" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q1', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q1" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q1', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q1" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q1', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q1" value="Agree" onclick="saveAndMarkAnswer('adaptability_q1', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q1" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q1', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>2. I quickly adjust my plans when circumstances change.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q2" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q2', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q2" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q2', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q2" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q2', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q2" value="Agree" onclick="saveAndMarkAnswer('adaptability_q2', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q2" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q2', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                       <p>3. I am open to new ideas and approaches.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q3" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q3', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q3" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q3', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q3" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q3', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q3" value="Agree" onclick="saveAndMarkAnswer('adaptability_q3', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q3" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q3', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>4. I handle unexpected challenges calmly and effectively.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q4" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q4', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q4" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q4', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q4" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q4', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q4" value="Agree" onclick="saveAndMarkAnswer('adaptability_q4', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q4" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q4', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>5.I seek out new experiences to broaden my perspective.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q5" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q5', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q5" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q5', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q5" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q5', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q5" value="Agree" onclick="saveAndMarkAnswer('adaptability_q5', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q5" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q5', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                       <p>6. I have a hard time making decisions when faced with difficult situations.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="adaptability_q6" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q6', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q6" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q6', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q6" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q6', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q6" value="Agree" onclick="saveAndMarkAnswer('adaptability_q6', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q6" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q6', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>7.I consider multiple options before making a decision.</p>
                     </div>
                     <ul class="options">
                            <li><label><input type="radio" name="adaptability_q7" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q7', 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="adaptability_q7" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q7', 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="adaptability_q7" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q7', 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="adaptability_q7" value="Agree" onclick="saveAndMarkAnswer('adaptability_q7', 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="adaptability_q7" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q7', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                         <p>8. I like to look at difficult situations from many different angles.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="adaptability_q8" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q8', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q8" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q8', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q8" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q8', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q8" value="Agree" onclick="saveAndMarkAnswer('adaptability_q8', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q8" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q8', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>9.I try to think about things from another person's point of view.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="adaptability_q9" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q9', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q9" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q9', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q9" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q9', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q9" value="Agree" onclick="saveAndMarkAnswer('adaptability_q9', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q9" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q9', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>10. I am capable of overcoming the difficulties in life that I face.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="adaptability_q10" value="Strongly Disagree" onclick="saveAndMarkAnswer('adaptability_q10', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q10" value="Disagree" onclick="saveAndMarkAnswer('adaptability_q10', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="adaptability_q10" value="Neutral" onclick="saveAndMarkAnswer('adaptability_q10', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="adaptability_q10" value="Agree" onclick="saveAndMarkAnswer('adaptability_q10', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="adaptability_q10" value="Strongly Agree" onclick="saveAndMarkAnswer('adaptability_q10', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>


                    <!-- Repeat for other questions -->
                </div>

                <div class="form-group">
                    <a href="/empathy" class="nav-button">Previous</a>
                    <input type="submit" value="Next" class="nav-button">
                </div>
            </form>
        </div>
    </div>

    <script>
// Save the current state of responses and category visibility
    function saveState() {
        // Save selected responses
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            if (option.checked) {
                localStorage.setItem(option.name, option.value);
            }
        });

        // Save visibility of categories
        const categories = document.querySelectorAll('.questions');
        categories.forEach(category => {
            localStorage.setItem(category.id + '_display', category.style.display);
        });

        // Save answered state of questions
        const answeredQuestions = document.querySelectorAll('.questions span.answered');
        answeredQuestions.forEach(question => {
            localStorage.setItem(question.id + '_answered', 'true');
        });

        // Save completed state of categories
        const completedCategories = document.querySelectorAll('.left li.completed');
        completedCategories.forEach(category => {
            localStorage.setItem(category.getAttribute('data-id') + '_completed', 'true');
        });
    }

    // Restore the state of responses, category visibility, and category completion
    function restoreState() {
        // Restore selected responses
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            const savedValue = localStorage.getItem(option.name);
            if (savedValue && option.value === savedValue) {
                option.checked = true;
            }
        });

        // Restore visibility of categories
        const categories = document.querySelectorAll('.questions');
        categories.forEach(category => {
            const display = localStorage.getItem(category.id + '_display');
            if (display) {
                category.style.display = display;
            }
        });

        // Restore answered state of questions
        const answeredQuestions = document.querySelectorAll('.questions span');
        answeredQuestions.forEach(question => {
            const isAnswered = localStorage.getItem(question.id + '_answered');
            if (isAnswered) {
                question.classList.add('answered');
            }
        });

        // Restore completed state of categories
        const categoriesList = document.querySelectorAll('.left li');
        categoriesList.forEach(category => {
            const isCompleted = localStorage.getItem(category.getAttribute('data-id') + '_completed');
            if (isCompleted) {
                category.classList.add('completed');
            }
        });
    }

    // Toggle visibility of questions under a specific category
    function toggleQuestions(categoryId) {
        const questionsDiv = document.getElementById(categoryId);
        const isVisible = questionsDiv.style.display === 'block';
        document.querySelectorAll('.questions').forEach(div => div.style.display = 'none');
        questionsDiv.style.display = isVisible ? 'none' : 'block';
        saveState(); // Save state when toggling visibility
    }

    // Save the answer and mark the question as answered
    function saveAndMarkAnswer(questionId, answer) {
        saveState(); // Save state immediately
        const questionSpan = document.getElementById(questionId);
        if (questionSpan) {
            questionSpan.classList.add('answered');
        }
    }

    // Mark the category as completed and save state
    function markCategoryAsCompleted(categoryId) {
        const categoryElement = document.querySelector(`.left li[data-id="${categoryId}"]`);
        if (categoryElement) {
            categoryElement.classList.add('completed');
            saveState(); // Save state when marking category as completed
        }
    }

    // Attach event listeners to buttons
    document.querySelectorAll('.nav-button').forEach(button => {
        button.addEventListener('click', function() {
            saveState(); // Save state when navigating
        });
    });

    // Call restoreState when the page loads
    window.onload = function() {
        restoreState();
    };

    // Example usage for marking categories as completed when navigating to the next category
    document.querySelector('input[type="submit"]').addEventListener('click', function() {
        const currentCategoryId = document.querySelector('.left li.active')?.getAttribute('data-id');
        if (currentCategoryId) {
            markCategoryAsCompleted(currentCategoryId);
        }
    });
    </script>
</body>
</html>
"""
# Display the HTML


from IPython.display import HTML

# HTML code as a string
html_communication= """
# HTML code as a string
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Assessment Page</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            height: 100vh;
            box-sizing: border-box;
            background-color: #f5f5f5;
        }
        .split {
            flex: 1;
            display: flex;
            padding: 20px;
            box-sizing: border-box;
        }
        .left {
            width: 20%;
            background-color: blueviolet;
            color: #fff;
            position: fixed;
            height: 100vh;
            overflow-y: auto;
            padding: 20px;
            text-decoration: none;
            display: inline-block;
        }
        .right {
            background-color: #fff;
            padding-left: 40px;
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            margin-left: 20%;
        }
        .left h3, .right h2 {
            margin: 0;
            padding: 10px 0;
            border-bottom: 2px solid #fff;
            text-align: center;
        }
        .left ul {
            list-style-type: none;
            padding: 0;
            margin: 0;
            color: #fff;
            text-decoration: none;
        }
        .left li {
            padding: 10px;
            border-bottom: 1px solid #555;
        }
        .left a {
            color: white;
            text-decoration: none;
            display: block;
        }
        .left li:hover {
            background-color: #444;
        }
        .question-container {
            flex-grow: 1;
            display: flex;
            flex-direction: column;
            justify-content: center;
        }
        .question {
            margin-bottom: -2%;
        }
        .options {
            list-style-type: none;
            padding: 0;
        }
        .options li {
            margin: 10px 0;
        }
        .options input {
            margin-right: 10px;
        }
        .form-group {
            display: flex;
            justify-content: space-between;
            padding: 20px 0;
        }
        .nav-button {
            padding: 10px 20px;
            background-color: #28a745;
            border: none;
            border-radius: 4px;
            color: white;
            font-size: 16px;
            cursor: pointer;
            text-decoration: none;
        }
        .nav-button:hover {
            background-color: #218838;
        }
        .questions {
            display: none;
            padding: 10px;
            margin-left: 20px;
            margin-top: 10px;
        }
        .questions span {
            display: inline-block;
            width: 25px;
            height: 30px;
            line-height: 30px;
            margin: 5px;
            text-align: center;
            background-color: red;
            color: white;
            border-radius: 50%;
            cursor: pointer;
        }
        .questions span.answered {
            background-color: green;
        }
    </style>
</head>
<body>
    <div class="split left">
        <h3>Categories</h3>
        <ul>
            <li onclick="toggleQuestions('empathy')">Empathy</li>
            <div id="empathy" class="questions">
            </div>
            <li onclick="toggleQuestions('adaptability')">Adaptability</li>
            <div id="adaptability" class="questions">
            </div>
            <li onclick="toggleQuestions('communication')">Communication</li>
            <div id="communication" class="questions">
                <span id="communication_q1">1</span><span id="communication_q2">2</span><span id="communication_q3">3</span><span id="communication_q4">4</span><span id="communication_q5">5</span>
                <span id="communication_q6">6</span><span id="communication_q7">7</span><span id="communication_q8">8</span><span id="communication_q9">9</span><span id="communication_q10">10</span>
            </div>
        </ul>
    </div>

    <div class="split right">
        <div>
            <h2>Communication Questions</h2>
            <form action="/communication" method="POST">
                <div class="question-container">
                    <!-- Questions and options go here -->
                    <!-- Example for the first question: -->
                    <div class="question">
                        <p>1. I clearly express my thoughts and ideas.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q1" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q1', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q1" value="Disagree" onclick="saveAndMarkAnswer('communication_q1', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q1" value="Neutral" onclick="saveAndMarkAnswer('communication_q1', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q1" value="Agree" onclick="saveAndMarkAnswer('communication_q1', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q1" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q1', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>2. I effectively communicate in both verbal and written forms.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q2" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q2', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q2" value="Disagree" onclick="saveAndMarkAnswer('communication_q2', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q2" value="Neutral" onclick="saveAndMarkAnswer('communication_q2', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q2" value="Agree" onclick="saveAndMarkAnswer('communication_q2', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q2" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q2', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                       <p>3. I am actively seek feedback to imporve my communication skills.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q3" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q3', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q3" value="Disagree" onclick="saveAndMarkAnswer('communication_q3', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q3" value="Neutral" onclick="saveAndMarkAnswer('communication_q3', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q3" value="Agree" onclick="saveAndMarkAnswer('communication_q3', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q3" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q3', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>4. I adjust my communication style based on the audience.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q4" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q4', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q4" value="Disagree" onclick="saveAndMarkAnswer('communication_q4', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q4" value="Neutral" onclick="saveAndMarkAnswer('communication_q4', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q4" value="Agree" onclick="saveAndMarkAnswer('communication_q4', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q4" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q4', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                        <p>5.I can resolve conflicts through effective communication.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q5" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q5', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q5" value="Disagree" onclick="saveAndMarkAnswer('communication_q5', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q5" value="Neutral" onclick="saveAndMarkAnswer('communication_q5', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q5" value="Agree" onclick="saveAndMarkAnswer('communication_q5', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q5" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q5', 'Strongly Agree')"> Strongly Agree</label></li>
                    </ul>
                    <div class="question">
                       <p>6. I am open minded and i am willing to change my viewpoint based on the vaild opinion of others.</p>
                    </div>
                    <ul class="options">
                        <li><label><input type="radio" name="communication_q6" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q6', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q6" value="Disagree" onclick="saveAndMarkAnswer('communication_q6', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q6" value="Neutral" onclick="saveAndMarkAnswer('communication_q6', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q6" value="Agree" onclick="saveAndMarkAnswer('communication_q6', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q6" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q6', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>7. I prepare for all communications and think things through before I speak.</p>
                     </div>
                     <ul class="options">
                            <li><label><input type="radio" name="communication_q7" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q7', 'Strongly Disagree')"> Strongly Disagree</label></li>
                            <li><label><input type="radio" name="communication_q7" value="Disagree" onclick="saveAndMarkAnswer('communication_q7', 'Disagree')"> Disagree</label></li>
                            <li><label><input type="radio" name="communication_q7" value="Neutral" onclick="saveAndMarkAnswer('communication_q7', 'Neutral')"> Neutral</label></li>
                            <li><label><input type="radio" name="communication_q7" value="Agree" onclick="saveAndMarkAnswer('communication_q7', 'Agree')"> Agree</label></li>
                            <li><label><input type="radio" name="communication_q7" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q7', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                         <p>8. I make a conscious effort to match my body language to the message I want to convey.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="communication_q8" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q8', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q8" value="Disagree" onclick="saveAndMarkAnswer('communication_q8', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q8" value="Neutral" onclick="saveAndMarkAnswer('communication_q8', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q8" value="Agree" onclick="saveAndMarkAnswer('communication_q8', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q8" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q8', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>9. I don't think that my opinion is the most important point in the conversation.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="communication_q9" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q9', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q9" value="Disagree" onclick="saveAndMarkAnswer('communication_q9', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q9" value="Neutral" onclick="saveAndMarkAnswer('communication_q9', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q9" value="Agree" onclick="saveAndMarkAnswer('communication_q9', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q9" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q9', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>
                     <div class="question">
                        <p>10. I don't think that my opinion is the most important point in the conversation.</p>
                     </div>
                     <ul class="options">
                        <li><label><input type="radio" name="communication_q10" value="Strongly Disagree" onclick="saveAndMarkAnswer('communication_q10', 'Strongly Disagree')"> Strongly Disagree</label></li>
                        <li><label><input type="radio" name="communication_q10" value="Disagree" onclick="saveAndMarkAnswer('communication_q10', 'Disagree')"> Disagree</label></li>
                        <li><label><input type="radio" name="communication_q10" value="Neutral" onclick="saveAndMarkAnswer('communication_q10', 'Neutral')"> Neutral</label></li>
                        <li><label><input type="radio" name="communication_q10" value="Agree" onclick="saveAndMarkAnswer('communication_q10', 'Agree')"> Agree</label></li>
                        <li><label><input type="radio" name="communication_q10" value="Strongly Agree" onclick="saveAndMarkAnswer('communication_q10', 'Strongly Agree')"> Strongly Agree</label></li>
                     </ul>


                    <!-- Repeat for other questions -->
                </div>

                <div class="form-group">
                    <a href="/adaptability" class="nav-button">Previous</a>
                    <input type="submit" value="Next" class="nav-button">
                </div>
            </form>
        </div>
    </div>

    <script>

    // Toggle visibility of question categories
    function toggleQuestions(categoryId) {
        const questionsDiv = document.getElementById(categoryId);
        const isVisible = questionsDiv.style.display === 'block';

        // Hide all other question divs
        document.querySelectorAll('.questions').forEach(div => div.style.display = 'none');

        // Toggle display of the selected category
        questionsDiv.style.display = isVisible ? 'none' : 'block';

        // Mark the category as answered if it becomes visible
        if (!isVisible) {
            markCategoryAsAnswered(categoryId);
        }
    }

    // Save the responses and the state of span buttons
    function saveResponses() {
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            if (option.checked) {
                localStorage.setItem(option.name, option.value);
            }
        });

        const answeredSpans = document.querySelectorAll('.options span.answered');
        answeredSpans.forEach(span => {
            localStorage.setItem(span.id, 'answered');
        });
    }

    // Restore the saved responses and span button states
    function restoreResponses() {
        const options = document.querySelectorAll('.options input[type="radio"]');
        options.forEach(option => {
            const savedValue = localStorage.getItem(option.name);
            if (savedValue && option.value === savedValue) {
                option.checked = true;
                markQuestionAsAnswered(option.name);
                markCategoryAsAnsweredFromQuestion(option.name);
            }
        });

        const answeredSpans = document.querySelectorAll('.options span');
        answeredSpans.forEach(span => {
            const savedState = localStorage.getItem(span.id);
            if (savedState === 'answered') {
                span.classList.add('answered');
            }
        });
    }

    // Save responses and mark the span button as answered
    function saveAndMarkAnswer(questionId, answer) {
        saveResponses(); // Save the responses and span states immediately
        const questionSpan = document.getElementById(questionId);
        if (questionSpan) {
            questionSpan.classList.add('answered');
        }
    }

    // Save responses and navigate to the previous page
    function saveResponsesAndNavigate(url) {
        saveResponses(); // Save responses before navigation
        window.location.href = url;
    }

    // Mark the category as answered
    function markCategoryAsAnswered(categoryId) {
        const categoryElement = document.querySelector(`.left li:nth-child(${getCategoryIndex(categoryId)})`);
        if (categoryElement) {
            categoryElement.classList.add('answered');
        }
    }

    // Get the index of the category
    function getCategoryIndex(categoryId) {
        const categories = ['empathy', 'adaptability', 'communication'];
        return categories.indexOf(categoryId) + 1;
    }

    // Mark the question as answered
    function markQuestionAsAnswered(questionName) {
        const questionSpan = document.getElementById(questionName);
        if (questionSpan) {
            questionSpan.classList.add('answered');
        }
    }

    // Mark the category as answered based on the question
    function markCategoryAsAnsweredFromQuestion(questionName) {
        const questionNumber = questionName.replace('adaptability_q', '');
        const categoryId = getCategoryFromQuestion(questionNumber);
        markCategoryAsAnswered(categoryId);
    }

    // Get the category based on the question number
    function getCategoryFromQuestion(questionNumber) {
        if (questionNumber <= 10) {
            return 'empathy';
        } else if (questionNumber <= 10) {
            return 'adaptability';
        } else {
            return 'communication';
        }
    }

    // Event listener for the Previous button
    document.addEventListener('DOMContentLoaded', function() {
        // Restore responses when the page loads
        restoreResponses();

        // Add event listener to the Previous button
        const prevButton = document.querySelector('.form-group .nav-button');
        if (prevButton) {
            prevButton.addEventListener('click', function(event) {
                event.preventDefault(); // Prevent default link behavior
                saveResponsesAndNavigate(this.getAttribute('href')); // Navigate after saving
            });
        }
    });

    // Call restoreResponses when the page loads
    window.onload = function() {
        restoreResponses();
    };
</script>

</body>
</html>
"""
# Display the HTML
HTML(html_communication)

# Display the HTML

from IPython.display import HTML

# HTML code as a string
html_thank = """

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Thank You</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background: linear-gradient(to right, #2e096e, #4af1e4, #d365fb);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
            background-size: cover;
        }

        .container {
            background: white;
            justify-content: center;
            display: flex;
            flex-direction: column;
            padding: 2rem;
            border-radius: 10px;
            box-shadow: 0 14px 28px rgba(0, 0, 0, 0.25),
                        0 10px 10px rgba(0, 0, 0, 0.22);
            position: center;
            width: 780px;
            max-width: 200%;
            min-height: 600px;
        }

        .container h1 {
            text-align: center;
            color: #333;
            margin-bottom: 20px;
        }

        .container p {
            color: #333; /* Change the color here */
            margin-bottom: 40px;
        }

        .container a {
            bottom: 20px;
            display: inline-block;
            padding: 10px 20px;
            background-color: #28a745;
            color: #fff;
            text-decoration: none;
            border-radius: 4px;
            transition: background-color 0.3s ease;
        }

        .container a:hover {
            background-color: #218838;
        }
    </style>
</head>
<body>
    <div class="container">
        <br><br><br><h1>Thank You!</h1><br><br>
        <p>We appreciate your participation in the self-evaluation. Your insights are valuable and will help you in your journey of personal growth.
        We encourage you to reflect on your results and use them as a guide for setting meaningful goals.
        <br>If you have any questions or need further assistance, feel free to reach out to us.</p>
        <p>Thank you once again for taking this important step in your self-discovery journey. We wish you the best in achieving your aspirations and living your best life.</p><br><br>
        <br><br>
        <center><a href="{{ url_for('index') }}">Return to Home</a></center>
        <br><br><br>
        <p>Contact info ⓘ<br><br>☎︎  @  ⓕ  𝕏</p>
    </div>
</body>
</html>

"""

# Display the HTML
HTML(html_thank)


@app.route('/')
def start():
    return render_template_string(html_start)

@app.route('/index')
def index():
    return render_template_string(html_index)

@app.route('/description')
def description():
    return render_template_string(html_description)


@app.route('/signUp', methods=['POST'])
def signUp():
    name = request.form['username']
    email = request.form['email']
    password = request.form['password']
    # Here you should save the user to the database
    return redirect(url_for('index'))

@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']
    # Here you should verify the username and password
    # If login is successful, redirect to the description page
    return redirect(url_for('description'))

@app.route('/resetpassword')
def resetpassword():
    # Your logic here
    return render_template_string('resetpassword.html')



@app.route('/empathy', methods=['GET', 'POST'])
def empathy():
    if request.method == 'POST':
        for key, value in request.form.items():
            session[key] = value
        return redirect(url_for('adaptability'))
    return render_template_string(html_empathy)

@app.route('/adaptability', methods=['GET', 'POST'])
def adaptability():
    if request.method == 'POST':
        for key, value in request.form.items():
            session[key] = value
        return redirect(url_for('communication'))
    return render_template_string(html_adaptability)

@app.route('/communication', methods=['GET', 'POST'])
def communication():
    if request.method == 'POST':
        for key, value in request.form.items():
            session[key] = value
        return redirect(url_for('dashboard'))
    return render_template_string(html_communication)

logging.basicConfig(level=logging.DEBUG)



def process_responses(responses, categories):
    scores = {'empathy': 0, 'adaptability': 0, 'communication': 0}
    response_values = {
        'Strongly Disagree': 1,
        'Disagree': 2,
        'Neutral': 3,
        'Agree': 4,
        'Strongly Agree': 5,
    }
    for question, response in responses.items():
        category = categories.get(question)
        if category in scores:
            scores[category] += response_values.get(response, 0)
    return scores

def create_plot(data, title):
    categories = list(data.keys())
    values = list(data.values())

    plt.figure(figsize=(10, 5))
    bars = plt.bar(categories, values, color=['blue', 'green', 'red'])
    plt.title(title)
    plt.xlabel('Category')
    plt.ylabel('Score')

    # plt.ylim(0, 6)  # Ensure the Y-axis starts at 0 and goes slightly above 5
    # plt.yticks([0, 1, 3, 4, 5])

    # Adding labels to the bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center')

    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plt.close()

    # Encode the image in base64
    return base64.b64encode(img.getvalue()).decode('utf-8')


class Score:
    @staticmethod
    def evaluate_empathy(score):

        if 40 <= score <= 50:
            return "Highly Empathetic:You are compassionate and able to connect deeply with people."
        elif 30<= score <= 40:
            return "Moderately Empathetic:You understand others' feelings well but may not always feel them deeply."
        elif 20 <= score <= 30:
            return "Average Empathy: You can relate to others' feelings but may not always be fully attuned to them."
        elif 10 <= score <= 20:
            return "Low Empathy: You might find it challenging to understand and share the feelings of others."
        elif 5 <= score <= 10:
            return "Very Low Empathy: Developing greater emotional awareness can help you connect better with others."
        else:
            return "Invalid score"

    @staticmethod
    def evaluate_adaptability(score):
        if 40 <= score <= 50:
            return "Highly Adaptable:  You are flexible, resilient, and thrive in dynamic environments."
        elif 30 <= score <= 40:
            return "Moderately Adaptable: You're adaptable and manage change well, with occasional challenges."
        elif 20 <= score <= 30:
            return "Average Adaptability: You manage changes adequately but may feel uncomfortable with significant or sudden shifts."
        elif 10 <= score <= 20:
            return "Low Adaptability: Working on being more open to new experiences could help improve your adaptability."
        elif 5 <= score <= 10:
            return "Very Low Adaptability: Developing greater flexibility and resilience can help you navigate changes more effectively."
        else:
            return "Invalid score"

    @staticmethod
    def evaluate_communication(score):
        if 40 <= score <= 50:
            return "Excellent Communication: You express yourself clearly, listen well, and effectively resolve conflicts."
        elif 30 <= score <= 40:
            return "Good Communication: You generally express yourself clearly and handle most interactions well."
        elif 20 <= score <= 30:
            return "Average Communication: You can convey your thoughts but may struggle with clarity or listening at times."
        elif 10 <= score <= 20:
            return "Poor Communication: Improving your listening skills and clarity can help."
        elif 5 <= score <= 10:
            return "Very Poor Communication: Working on your listening and expression skills can greatly benefit your interactions."
        else:
            return "Invalid score"
@app.route('/dashboard', methods=(['GET', 'POST']))
def dashboard():
    # Extract stored responses from session
    responses = {key: value for key, value in session.items() if key.startswith('q') or key.startswith('adaptability_q') or key.startswith('communication_q')}
    print(f"Responses: {responses}")   # Debug: Print the user responses

    # Define categories mapping for questions
    categories = {
        'q1': 'empathy', 'q2': 'empathy', 'q3': 'empathy', 'q4': 'empathy', 'q5': 'empathy',
        'q6': 'empathy', 'q7': 'empathy', 'q8': 'empathy', 'q9': 'empathy', 'q10': 'empathy',
        'adaptability_q1': 'adaptability', 'adaptability_q2': 'adaptability', 'adaptability_q3': 'adaptability', 'adaptability_q4': 'adaptability',
        'adaptability_q5': 'adaptability', 'adaptability_q6': 'adaptability', 'adaptability_q7': 'adaptability', 'adaptability_q8': 'adaptability',
        'adaptability_q9': 'adaptability', 'adaptability_q10': 'adaptability',
        'communication_q1': 'communication', 'communication_q2': 'communication', 'communication_q3': 'communication', 'communication_q4': 'communication',
        'communication_q5': 'communication', 'communication_q6': 'communication', 'communication_q7': 'communication', 'communication_q8': 'communication',
        'communication_q9': 'communication', 'communication_q10': 'communication'
    }

    # Process the responses and create the plot
    processed_data = process_responses(responses, categories)
    print(f"Processed Scores: {processed_data}")  # Debug: Print the processed scores

    # Extract individual category scores
    empathy_score = processed_data['empathy']
    adaptability_score = processed_data['adaptability']
    communication_score = processed_data['communication']

    # Generate the plot URL after processing the data
    plot_url = create_plot(processed_data, 'Category Scores')

    # Generate descriptions for each category based on the scores
    empathy_description = Score.evaluate_empathy(empathy_score)
    adaptability_description = Score.evaluate_adaptability(adaptability_score)
    communication_description = Score.evaluate_communication(communication_score)

    print(f"Empathy Description: {empathy_description}")  # Debug: Print empathy description
    print(f"Adaptability Description: {adaptability_description}")  # Debug: Print adaptability description
    print(f"Communication Description: {communication_description}")  # Debug: Print communication description
    username = session.get('username', 'User')

    # Render the HTML with the plot, scores, and descriptions
    return render_template_string(html_dashboard,
                                  plot_url=plot_url,
                                  empathy_score=empathy_score,
                                  adaptability_score=adaptability_score,
                                  communication_score=communication_score,
                                  empathy_description=empathy_description,
                                  adaptability_description=adaptability_description,
                                  communication_description=communication_description)
    return redirect(url_for('thank'))



# from io import BytesIO
# from flask import send_file, session
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas

def create_pdf(empathy_score, empathy_description, adaptability_score, adaptability_description, communication_score, communication_description):
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter

    # Set the title with a larger font size
    c.setFont("Helvetica-Bold", 16)
    c.drawString(70, height - 70, "Report")

    # Set font for the body text
    c.setFont("Helvetica", 12)

    # Empathy Section
    y = height - 100
    c.drawString(70, y, "Empathy:")
    c.drawString(70, y - 20, f"Score: {empathy_score}")
    text_object = c.beginText(70, y - 40)
    text_object.setFont("Helvetica", 12)
    text_object.textLines(empathy_description)
    c.drawText(text_object)

    # Adaptability Section
    y -= 100
    c.drawString(70, y, "Adaptability:")
    c.drawString(70, y - 20, f"Score: {adaptability_score}")
    text_object = c.beginText(70, y - 40)
    text_object.setFont("Helvetica", 12)
    text_object.textLines(adaptability_description)
    c.drawText(text_object)

    # Communication Section
    y -= 100
    c.drawString(70, y, "Communication:")
    c.drawString(70, y - 20, f"Score: {communication_score}")
    text_object = c.beginText(70, y - 40)
    text_object.setFont("Helvetica", 12)
    text_object.textLines(communication_description)
    c.drawText(text_object)

    c.showPage()
    c.save()

    buffer.seek(0)
    return buffer




@app.route('/download_report')
def download_report():
    # Extract stored responses from session
    responses = {key: value for key, value in session.items() if key.startswith(('q', 'adaptability_q', 'communication_q'))}

    # Define categories mapping for questions
    categories = {
        'q1': 'empathy', 'q2': 'empathy', 'q3': 'empathy', 'q4': 'empathy', 'q5': 'empathy',
        'q6': 'empathy', 'q7': 'empathy', 'q8': 'empathy', 'q9': 'empathy', 'q10': 'empathy',
        'adaptability_q1': 'adaptability', 'adaptability_q2': 'adaptability', 'adaptability_q3': 'adaptability', 'adaptability_q4': 'adaptability',
        'adaptability_q5': 'adaptability', 'adaptability_q6': 'adaptability', 'adaptability_q7': 'adaptability', 'adaptability_q8': 'adaptability',
        'adaptability_q9': 'adaptability', 'adaptability_q10': 'adaptability',
        'communication_q1': 'communication', 'communication_q2': 'communication', 'communication_q3': 'communication', 'communication_q4': 'communication',
        'communication_q5': 'communication', 'communication_q6': 'communication', 'communication_q7': 'communication', 'communication_q8': 'communication',
        'communication_q9': 'communication', 'communication_q10': 'communication'
    }

    # Define calculate_score function
    def calculate_score(category, responses, categories):
        scoring = {'Agree': 4, 'Strongly Agree': 5, 'Neutral': 3, 'Disagree': 2, 'Strongly Disagree': 1}
        category_questions = [key for key in responses if categories.get(key) == category]
        score = sum(scoring.get(responses.get(q, 'Neutral'), 3) for q in category_questions)
        return score

    # Calculate the scores for each category
    empathy_score = calculate_score('empathy', responses, categories)
    adaptability_score = calculate_score('adaptability', responses, categories)
    communication_score = calculate_score('communication', responses, categories)

    # Generate descriptions for each category based on the scores
    empathy_description = Score.evaluate_empathy(empathy_score)
    adaptability_description = Score.evaluate_adaptability(adaptability_score)
    communication_description = Score.evaluate_communication(communication_score)

    # Create PDF report
    pdf_buffer = create_pdf(empathy_score, empathy_description, adaptability_score, adaptability_description, communication_score, communication_description)

    # Return PDF file for download
    return send_file(pdf_buffer, as_attachment=True, download_name='report.pdf', mimetype='application/pdf')





html_dashboard="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        h1, h2 {
            color: #333;
        }
        .btn {
            display: inline-block;
            padding: 10px 20px;
            margin: 10px 0;
            font-size: 16px;
            color: #fff;
            background-color: #007bff;
            text-decoration: none;
            border-radius: 5px;
            text-align: center;
        }
        .btn:hover {
            background-color: #0056b3;
        }
    </style>
</head>
<body>
    <h1>Dashboard</h1>
    <img src="data:image/png;base64,{{ plot_url }}" alt="Plot">

    <h2>Empathy</h2>
    <p><strong>Score:</strong> {{ empathy_score }}</p>
    <p>{{ empathy_description }}</p>

    <h2>Adaptability</h2>
    <p><strong>Score:</strong> {{ adaptability_score }}</p>
    <p>{{ adaptability_description }}</p>

    <h2>Communication</h2>
    <p><strong>Score:</strong> {{ communication_score }}</p>
    <p>{{ communication_description }}</p>

    <a href="{{ url_for('download_report') }}" class="btn">Download Report</a>

    <!-- Complete Button -->
    <form action="{{ url_for('thank_you') }}" method="post">
        <button type="submit" class="btn">Complete</button>
    </form>

</body>
</html>
"""
@app.route('/thank_you', methods=['POST'])
def thank_you():
    return render_template_string(html_thank)




if __name__ == '__main__':
    ngrok.set_auth_token("2lvbYyZyAJHVmI2YhTGNSKFkCcM_546qvqc4SDSoJf5JGG4VC")  # Replace with your actual ngrok authtoken
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
    app.run(port=5000)

 * ngrok tunnel "NgrokTunnel: "https://f2c8-34-125-110-240.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:11:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:11:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:12:00] "GET /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:12:32] "POST /signUp HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:12:32] "GET /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:13:16] "GET /index/description HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 16:13:19] "GET /index/description HTTP/1.1" 404 -
